In [0]:
from pyspark.sql.functions import rand, round, col, lit, sqrt, sin, cos, atan2, radians

# STEP 1: Load Sales Report Table
df = spark.table("workspace.salesdata.sale_report")
df.printSchema()
df.show(5)


root
 |-- index: long (nullable = true)
 |-- SKU Code: string (nullable = true)
 |-- Design No.: string (nullable = true)
 |-- Stock: double (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Color: string (nullable = true)

+-----+-------------+----------+-----+-------------+----+-----+
|index|     SKU Code|Design No.|Stock|     Category|Size|Color|
+-----+-------------+----------+-----+-------------+----+-----+
|    0|  AN201-RED-L|     AN201|  5.0|AN : LEGGINGS|   L|  Red|
|    1|  AN201-RED-M|     AN201|  5.0|AN : LEGGINGS|   M|  Red|
|    2|  AN201-RED-S|     AN201|  3.0|AN : LEGGINGS|   S|  Red|
|    3| AN201-RED-XL|     AN201|  6.0|AN : LEGGINGS|  XL|  Red|
|    4|AN201-RED-XXL|     AN201|  3.0|AN : LEGGINGS| XXL|  Red|
+-----+-------------+----------+-----+-------------+----+-----+
only showing top 5 rows


 STEP 2: Generate Synthetic Product Metadata

In [0]:
# Extract distinct SKU Code
df_skus = df.select("`SKU Code`").distinct()

# Generate synthetic weight, dimensions, cost price, and profit
df_products = df_skus \
    .withColumn("Weight_kg", round(rand() * 4 + 1, 2)) \
    .withColumn("Length_cm", round(rand() * 40 + 10, 2)) \
    .withColumn("Width_cm", round(rand() * 30 + 5, 2)) \
    .withColumn("Height_cm", round(rand() * 20 + 5, 2)) \
    .withColumn("Cost_Price", round(rand() * 500 + 200, 2)) \
    .withColumn("Profit", round(rand() * 300, 2))

df_products.show(5)


+-------------+---------+---------+--------+---------+----------+------+
|     SKU Code|Weight_kg|Length_cm|Width_cm|Height_cm|Cost_Price|Profit|
+-------------+---------+---------+--------+---------+----------+------+
|   BTM008-XXL|     3.15|    49.98|   10.65|    23.54|     281.7|  4.95|
|   J0166-DR-L|     4.38|    10.99|    9.62|    23.52|    467.59|119.67|
|  J0297-TP-XS|      2.1|    38.38|   28.31|    11.89|    265.53|135.72|
|JNE3426-KR-XL|     2.16|    46.86|    5.54|     7.67|    237.99|219.64|
|  J0112-TP-XS|     4.27|    11.55|   31.78|    15.99|    662.43| 16.16|
+-------------+---------+---------+--------+---------+----------+------+
only showing top 5 rows


STEP 3: Generate Random Lat/Lon Coordinates (Customer + Supplier)

In [0]:
# Add synthetic lat/lon for shipping FROM and TO
df_geo = df \
    .withColumn("Customer_Lat", round(rand() * 180 - 90, 6)) \
    .withColumn("Customer_Lon", round(rand() * 360 - 180, 6)) \
    .withColumn("Supplier_Lat", round(rand() * 180 - 90, 6)) \
    .withColumn("Supplier_Lon", round(rand() * 360 - 180, 6))


 STEP 4: Join with Product Metadata and Calculate Shipping Cost

In [0]:
from pyspark.sql.functions import (
    radians, sin, cos, atan2, sqrt,
    col, round, lit, when
)

#  1. Join orders + product dimensions + geo info
df_joined = df_geo.join(df_products, on="SKU Code", how="inner")

# 2. Haversine formula to calculate distance (km)
df_joined = df_joined \
    .withColumn("dLat", radians(col("Customer_Lat") - col("Supplier_Lat"))) \
    .withColumn("dLon", radians(col("Customer_Lon") - col("Supplier_Lon"))) \
    .withColumn(
        "a",
        sin(col("dLat") / 2) ** 2 +
        cos(radians(col("Supplier_Lat"))) * cos(radians(col("Customer_Lat"))) *
        sin(col("dLon") / 2) ** 2
    ) \
    .withColumn("c", 2 * atan2(sqrt(col("a")), sqrt(1 - col("a")))) \
    .withColumn("Distance_km", round(6371 * col("c"), 2))  # Earth's radius = 6371 km

# 3. Volumetric weight (Length × Width × Height / 5000)
df_joined = df_joined.withColumn(
    "Volumetric_Weight",
    round((col("Length_cm") * col("Width_cm") * col("Height_cm")) / 5000, 2)
)

# 4. Effective shipping weight = max of physical vs volumetric
df_joined = df_joined.withColumn(
    "Effective_Weight",
    when(col("Volumetric_Weight") > col("Weight_kg"), col("Volumetric_Weight"))
    .otherwise(col("Weight_kg"))
)
df_joined = df_joined.withColumn("Effective_Weight", round(col("Effective_Weight"), 2))

# 5. Shipping cost = ₹50 base + ₹5/km + ₹10/kg
df_joined = df_joined.withColumn(
    "Shipping_Cost",
    round(lit(50) + col("Distance_km") * 5 + col("Effective_Weight") * 10, 2)
)

# Optional preview
df_joined.select(
    "SKU Code", "Distance_km", "Weight_kg", "Volumetric_Weight",
    "Effective_Weight", "Shipping_Cost"
).show(10)


+----------------+-----------+---------+-----------------+----------------+-------------+
|        SKU Code|Distance_km|Weight_kg|Volumetric_Weight|Effective_Weight|Shipping_Cost|
+----------------+-----------+---------+-----------------+----------------+-------------+
|     AN201-RED-L|    4024.89|     3.35|             1.74|            3.35|     20207.95|
|     AN201-RED-M|    9308.83|     4.02|              0.5|            4.02|     46634.35|
|     AN201-RED-S|   12130.33|     4.95|             0.35|            4.95|     60751.15|
|    AN201-RED-XL|    2148.52|      4.5|             2.75|             4.5|      10837.6|
|   AN201-RED-XXL|   11950.15|     1.96|              1.1|            1.96|     59820.35|
|  AN202-ORANGE-L|   12674.87|     1.82|             5.29|            5.29|     63477.25|
|  AN202-ORANGE-M|   10498.82|     4.92|             1.86|            4.92|      52593.3|
|  AN202-ORANGE-S|    7226.25|     2.95|             1.87|            2.95|     36210.75|
| AN202-OR

In [0]:
df_joined = df_joined \
    .withColumnRenamed("SKU Code", "SKU_Code") \
    .withColumnRenamed("Design No.", "Design_No")
    
   


STEP 6: Save Final Shipping Estimates to Delta Table

In [0]:
df_joined.write.format("delta").mode("overwrite").saveAsTable("workspace.salesdata.shipping_cost_estimates")
